In [ ]:
import os
from pathlib import Path
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn import metrics
import gc
from tqdm import tqdm
tqdm.pandas()
import time

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D, CuDNNLSTM, concatenate
from keras.layers import Bidirectional, GlobalMaxPool1D, Dropout, SpatialDropout1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers, models

import tensorflow as tf

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
# some config values 
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 70 # max number of words in a question to use

S_DROPOUT = 0.4
DROPOUT = 0.1

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
#sub = pd.read_csv(Path('../input/sample_submission.csv'))

In [ ]:
# preprocessing
print("Preprocessing")
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

train["question_text"] = train["question_text"].progress_apply(lambda x: clean_text(x))
test["question_text"] = test["question_text"].progress_apply(lambda x: clean_text(x))

In [ ]:
# create a tokenizer 
print("Creating tokenizer")
token = text.Tokenizer(num_words=max_features)
token.fit_on_texts(train['question_text'])
word_index = token.word_index

In [ ]:
# loading word embeddings:
print("Loading GloVe")
EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8"))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = token.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_1 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_1[i] = embedding_vector

del embeddings_index; gc.collect() 

In [ ]:
print("Loading wiki-news-300d-1M")
EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8") if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = token.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_2 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_2[i] = embedding_vector
        
del embeddings_index; gc.collect()   

In [ ]:
print("Loading paragram_300_sl999")
EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = token.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_3 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_3[i] = embedding_vector

del embeddings_index; gc.collect()

In [ ]:
print("Loading word2Vec")
# https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go
from gensim.models import KeyedVectors

EMBEDDING_FILE = '../input/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
embeddings_index = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

word_index = token.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_4 = (np.random.rand(nb_words, embed_size) - 0.5) / 5.0
for word, i in word_index.items():
    if i >= max_features: continue
    if word in embeddings_index:
        embedding_vector = embeddings_index.get_vector(word)
        embedding_matrix_4[i] = embedding_vector
        
del embeddings_index; gc.collect() 

In [ ]:
embedding_matrix = np.concatenate((embedding_matrix_1, embedding_matrix_2, embedding_matrix_3, embedding_matrix_4), axis=1)  
del embedding_matrix_1, embedding_matrix_2, embedding_matrix_3, embedding_matrix_4
gc.collect()
np.shape(embedding_matrix)

In [ ]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((maxlen, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(max_features, 300*4, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

In [ ]:
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D

def create_cnn_v2():
    filter_sizes = [1,2,3,5]
    num_filters = 36

    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size * 4, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(S_DROPOUT)(x)
    x = Reshape((maxlen, embed_size * 4, 1))(x)

    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embed_size * 4),
                                     kernel_initializer='he_normal', activation='elu')(x)
        maxpool_pool.append(MaxPool2D(pool_size=(maxlen - filter_sizes[i] + 1, 1))(conv))

    z = Concatenate(axis=1)(maxpool_pool)   
    z = Flatten()(z)
    z = Dropout(DROPOUT)(z)

    outp = Dense(1, activation="sigmoid")(z)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
old_cnn = create_cnn()
cnn_v2_model = create_cnn_v2()

print(old_cnn.summary())
print(cnn_v2_model.summary())

#from keras.utils import plot_model
#plot_model(old_cnn, to_file='model.png')

In [ ]:
def train_and_evaluate__model(model, data_train, labels_train, data_valid, labels_valid, data_test, labels_test):
    start = time.time()
    model.fit(data_train, labels_train, batch_size=512, epochs=2, validation_data=(data_test, labels_test), verbose=0)
    end = time.time()
    print('Training time: ', end - start)
    pred_val_cnn_y = model.predict(data_valid, batch_size=1024)

    # finding best threshold with validation data 
    pred_val_y = pred_val_cnn_y
    thresholds = []
    for thresh in np.arange(0.1, 0.501, 0.01):
        thresh = np.round(thresh, 2)
        #print(type(labels_valid))
        res = metrics.f1_score(labels_valid, (pred_val_y > thresh).astype(int))
        thresholds.append([thresh, res])
        print("F1 score at threshold {0} is {1}".format(thresh, res))

    thresholds.sort(key=lambda x: x[1], reverse=True)
    best_thresh = thresholds[0][0]
    print("Best threshold: ", best_thresh)
    
    # 
    test_prediction = model.predict(data_test, batch_size=1024)
    pred_test_y = (test_prediction > best_thresh).astype(int)
    recall = sklearn.metrics.recall_score(labels_test, pred_test_y)
    print("Recall: %f" % recall)

    precision = sklearn.metrics.precision_score(labels_test, pred_test_y)
    print("Precision: %f" % precision)

    f1_score = sklearn.metrics.f1_score(labels_test, pred_test_y)
    print("F1-score: %f" % f1_score)

In [ ]:
# split the dataset into training and validation datasets 
print("Train/test split")
train_x, test_x, train_y, test_y = model_selection.train_test_split(train['question_text'], train['target'], test_size=0.1, random_state=4)

In [ ]:
# Splitting all the data into train and test set.
# (Check if tokenizer should only be using train split data)
# Using the data from the train split for the cross validation (train and validation split)

In [ ]:
# convert text to sequence of tokens and pad them to ensure equal length vectors 
print("Padding sentences")
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=maxlen)
test_seq_x = sequence.pad_sequences(token.texts_to_sequences(test_x), maxlen=maxlen)

In [ ]:
#train- data/labels
#validation data/labels
#test data/labels

# data_train, labels_train, data_valid, labels_valid

In [ ]:
# adding grwosth parameter for GPU memory allocation
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [ ]:
from sklearn.model_selection import StratifiedKFold
from keras import backend as K


K.clear_session()

n_folds = 10
# data, labels, header_info = load_data()
# data = sequence.pad_sequences(token.texts_to_sequences(train['question_text']), maxlen=maxlen)
data = train_seq_x
labels = np.asarray(train_y)
skf = StratifiedKFold(n_splits=n_folds, shuffle=True)
foo = []
bar = []
for i, (train, valid) in enumerate(skf.split(data, labels)):
    print("Running Fold", i+1, "/", n_folds)
    
    train_data = data[train]
    train_labels = labels[train]
    
    validation_data = data[valid]
    validation_labels = labels[valid]
    
    K.clear_session()
    model = None # Clearing the NN.
    model = create_cnn()
    train_and_evaluate__model(model, train_data, train_labels, validation_data, validation_labels, test_seq_x, np.asarray(test_y))
    del model; gc.collect()